Подготовка дат для дальнейшей работы

In [1]:
from datetime import datetime, timedelta

now = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
today = datetime.now().strftime('%Y_%m_%d')
today_begin = datetime.now().strftime('%Y-%m-%d 00:00:00.00')
last_3_days = (datetime.now() - timedelta(days = 2)).strftime('%Y-%m-%d 00:00:00.00')

Настройка логирования

In [2]:
import logging

for h in logging.root.handlers[:]:
    logging.root.removeHandler(h)

logging.basicConfig(
    filename=f'{today}.log',
    filemode = 'w',
    format='%(asctime)s %(name)s %(levelname)s: %(message)s',
    level=logging.INFO)

Работа с API

In [ ]:
import requests
from datetime import datetime, timedelta

api_url = "***********************"
# date format - '2025-11-01 12:46:47.860798'
logger = logging.getLogger(__name__)

def get_data(url = api_url, client = 'Skillfactory', client_key = 'M2MGWS', start = today_begin, end = now):
  params = {'client' : client, 'client_key' : client_key, 'start' : start, 'end' : end}
  logger.info(f'Отправляем запрос к API: url={url}, params={params}')
  try:
    r = requests.get(api_url, params = params)
    logger.info(f'Ответ API: status_code={r.status_code}')
    data = r.json()
    logger.info(f'Успешно получили данные, записей: {len(data)}')
    return data
  except requests.exceptions.RequestException as error:
    logger.exception('Ошибка при обращении к API')
    raise


data = get_data()

In [ ]:
import json

def create_dict_json(dict_):
  return json.loads(dict_['passback_params'].replace("'", '"'))
    
logger.info(f'Подготовка данных к загрузке в БД')
edited_data = []

for el in data:
  passback_params = create_dict_json(el)
  edited_el = {
        'user_id': el['lti_user_id'],
        'oauth_consumer_key': passback_params.get('oauth_consumer_key', ''),
        'lis_result_sourcedid': passback_params.get('lis_result_sourcedid', ''),
        'lis_outcome_service_url': passback_params.get('lis_outcome_service_url', ''),
        'is_correct': el['is_correct'],
        'attempt_type': el['attempt_type'],
        'created_at': el['created_at']
    }
  edited_data.append(edited_el)
# length_data = len(edited_data)
logger.info(f'Данные готовы к загрузке. В таблице {len(edited_data)} записей.')
edited_data

Работа с локальной БД

In [5]:
import psycopg2
from psycopg2.extras import execute_values

HOST = "localhost"        
PORT = 5432                
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "strongpass"

class DatabaseConnection:
    def __init__(self):
        self.connection = psycopg2.connect(
            host = HOST,
            port = PORT,
            database = DATABASE,
            user = USER,
            password = PASSWORD,       
        )
        self.cursor = self.connection.cursor()
    def create_table(self, name):
        query = f''' DROP TABLE IF EXISTS {name};
                     CREATE TABLE {name} 
                     (
                        user_id               TEXT,
                        oauth_consumer_key    TEXT,
                        lis_result_sourcedid  TEXT,
                        lis_outcome_service_url TEXT,
                        is_correct            TEXT,
                        attempt_type          TEXT,
                        created_at            TIMESTAMP 
                    )'''
        self.cursor.execute(query)
        self.connection.commit()
    def insert_data(self, name, data):
        query = f'''INSERT INTO {name} (
                        user_id,
                        oauth_consumer_key,
                        lis_result_sourcedid,
                        lis_outcome_service_url,
                        is_correct,
                        attempt_type,
                        created_at
                    )
                    VALUES %s
        '''
        values = [    
            (
                row['user_id'],
                row['oauth_consumer_key'],
                row['lis_result_sourcedid'],
                row['lis_outcome_service_url'],
                row['is_correct'],
                row['attempt_type'],
                row['created_at'],
            )
            for row in data]
        execute_values(self.cursor, query, values)
        self.connection.commit()
    def check_data(self, name):
        self.cursor.execute(f'SELECT COUNT(*) FROM {name};')
        length = self.cursor.fetchone()[0]
        return length
    def close_connection(self):
        self.cursor.close()
        self.connection.close()

In [6]:
name_data_to_db = 'our_data'
try:
    logger.info(f'Подключение к БД...')
    db_connection = DatabaseConnection()
    
    logger.info(f'Создание таблицы...')
    db_connection.create_table(name_data_to_db)
    
    logger.info(f'Добавление данных...')
    db_connection.insert_data(name_data_to_db, edited_data)
    length = db_connection.check_data(name_data_to_db)
    
    logger.info(f'Загрузка завершена. Загружено {length} строк(а). Отлючение от БД.')
    db_connection.close_connection()
    
except psycopg2.Error:
    logger.exception(f'Не удалось подключиться к БД.')
    raise

Удаление логов старше 3 дней

In [7]:
from pathlib import Path

def clean_old_logs(dir_: str = '', days: int = 3):
    cutoff_date = datetime.now().date() - timedelta(days=days)
    for file in Path(dir_).glob('*.log'):
        date_str = file.stem          # '2025_12_04'
        file_date = datetime.strptime(date_str, "%Y_%m_%d").date()
        if file_date < cutoff_date:
            file.unlink()  # удалить файл
clean_old_logs()

Расчёт метрик

In [8]:
cnt_corrects = 0
for el in edited_data:
    if el['is_correct'] == 1:
        cnt_corrects += 1
        perc_of_suc = str(round(cnt_corrects / len(edited_data) * 100, 2)) + '%'
cnt_corrects, perc_of_suc

(189, '22.42%')

In [9]:
unique_users = []
for el in edited_data:
    if el['user_id'] not in unique_users:
        unique_users.append(el['user_id'])
len(unique_users), len(edited_data)

(52, 843)

In [10]:
unique_urls = []
for el in edited_data:
    if el['lis_outcome_service_url'] not in unique_urls:
        unique_urls.append(el['lis_outcome_service_url'])
len(unique_urls), len(edited_data)
# unique_urls

(190, 843)

Работа с Google Sheets API

In [ ]:
!pip install gspread

In [12]:
import gspread
from google.oauth2.service_account import Credentials


SERVICE_ACCOUNT_FILE = 'python-project-480514-9a56a86417e1.json'
TABLE_URL = 'https://docs.google.com/spreadsheets/d/1BmOEc8ZBDlDV29P0alsriYNu_arnSXC-coyYzM3IL3Q/edit?usp=sharing'

def connect_to_GS(service_account_file_json = SERVICE_ACCOUNT_FILE):

    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive',
    ]
    
    creds = Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE,
        scopes=SCOPES,
    )
    client = gspread.authorize(creds)
    return client

def export_to_google_table(client, url = TABLE_URL, start = today_begin, end = now):
    
    sh = client.open_by_url(TABLE_URL)
    ws = sh.sheet1

    ws.update(range_name = 'A1', values= [['Промежуток времени', 'Количество попыток', 'Процент удачных попыток', 'Количество уникальных пользователей']])
    ws.append_row([start + '  —  ' + end, len(edited_data), perc_of_suc, len(unique_users)])

logger.info(f'Подключение к Google Sheets API...')
try:
    client = connect_to_GS()
except:
    logger.exception(f'Не удалось подключиться. Проверьте файл json.')
logger.info(f'Выгрузка метрик в таблицу...')
try:
    export_to_google_table(client)
    logger.info(f'Метрики загружены в таблицу по ссылке: {TABLE_URL}')
except gspread.exceptions.SpreadsheetNotFound as error:
    logger.exception(f'Не удалось выгрузить данные. Проверьте ссылку на таблицу')



Рассылка на почту

In [ ]:
import smtplib
import ssl
from email.message import EmailMessage

SMTP_SERVER = 'smtp.mail.ru' #smtp.mail.ru
SMTP_PORT = 465

SENDER_EMAIL = 'project_mail00@mail.ru'
SENDER_PASSWORD = '********************'

RECEIVERS = ['mail@mail.ru']
SUBJECT = 'Уведомление о выполненой работе!'
MESSAGE = 'Моя работа сделана на отлично, правда же?'


def send_email(to_email, subject, body):
  msg = EmailMessage()

  msg.set_content(body)
  msg['Subject'] = subject
  msg['From'] = SENDER_EMAIL
  msg['To'] = to_email

  context = ssl.create_default_context()

  logger.info(f'Оповещение отправлено на следующие почты: {RECEIVERS}')
    
  with smtplib.SMTP_SSL(SMTP_SERVER, SMTP_PORT, context=context) as server:

    server.login(SENDER_EMAIL, SENDER_PASSWORD)

    server.send_message(msg=msg)

send_email(RECEIVERS, SUBJECT, MESSAGE)